In [2]:
import json
import psycopg
from pgvector.psycopg import register_vector
import re
import os

conn = psycopg.connect(
    dbname="fpkg",
    user=os.environ.get("PGUSER"),
    password=os.environ.get("PGPASSWORD"),
    host="localhost",
    port="5431"
)

cursor = conn.cursor()

cursor.execute("LOAD 'age';")
cursor.execute("SET search_path = ag_catalog, \"$user\", public;")

conn.commit()

In [23]:
embedding = embeddings.embed_query("Paracétamol")

embedding_string = "[" + ','.join(str(x) for x in embedding) + "]"

In [30]:
# Finding the list of nodes to embed
query = f"""
    SELECT node_name, id, embedding
    FROM document_vectors
    ORDER BY embedding <-> %(vector)s
    LIMIT 5;
"""

try:
    cursor.execute(query, {'vector':embedding_string})
    results = cursor.fetchall()
    print(len(results))
    for res in results[:200]:
        # res = re.sub(r'::\w+$', '', res[0])
        print(res[0])

    vertex_labels = results
except Exception as e:
    print(e)
    conn.rollback()

5
"Paracétamol 150 mg suppositoire"
"Paracétamol 1 g comprimé"
"Paracétamol 300 mg suppositoire"
"Paracétamol 100 mg suppositoire"
"Paracétamol 600 mg suppositoire"
